In [1]:
# Cell 1: Import necessary libraries
import numpy as np
import os
import time
from pydrake.all import (
    DiagramBuilder, AddMultibodyPlantSceneGraph, Parser, RigidTransform, RotationMatrix,
    Role, MeshcatVisualizer, StartMeshcat, RationalForwardKinematics, CspaceFreePolytope,
    SeparatingPlaneOrder, Rgba, InverseKinematics,
    LinearEqualityConstraint, Sphere, Parallelism, AddDefaultVisualization, 
    ConnectPlanarSceneGraphVisualizer, IrisFromCliqueCoverOptions, 
    IrisInConfigurationSpaceFromCliqueCover, RandomGenerator, RobotDiagramBuilder, 
    SceneGraphCollisionChecker, MultibodyPlant, SceneGraph, 
    SolverOptions, CommonSolverOption, GeometrySet, ScsSolver
)
from pydrake.geometry.optimization import GraphOfConvexSetsOptions, HPolyhedron, VPolytope, Point, Hyperellipsoid
from pydrake.geometry.optimization import ConvexHull as DrakeConvexHull
from pydrake.planning import GcsTrajectoryOptimization
from pydrake.solvers import MathematicalProgram, Solve, MosekSolver
from pydrake.trajectories import CompositeTrajectory
from scipy.spatial import ConvexHull
import mcubes
from functools import partial
import matplotlib.pyplot as plt
from ciris_plant_visualizer import CIrisPlantVisualizer
from ipywidgets import widgets

# 1. Set up the Scene

In [3]:
# Replace DiagramBuilder with RobotDiagramBuilder
builder = RobotDiagramBuilder(time_step=0.0)
plant = builder.plant()
scene_graph = builder.scene_graph()
parser = Parser(plant, scene_graph)
parser.SetAutoRenaming(True)

# Add the robot
gripper = parser.AddModels(file_name="my_sdfs/wsg_2dof.sdf")[0]
cap = parser.AddModels(file_name="my_sdfs/bottle_cap.sdf")[0]
obstacle1 = parser.AddModels("my_sdfs/obstacle.sdf")[0]
# obstacle2 = parser.AddModels("my_sdfs/obstacle.sdf")[0]
obstacle3 = parser.AddModels("my_sdfs/obstacle.sdf")[0]

# Set welds
plant.WeldFrames(
    plant.world_frame(), 
    plant.GetFrameByName("base_link", cap),
    RigidTransform(RotationMatrix(), [0, 0, 0]))

# Weld the obstacle to the world frame (adjust pose as needed)
obstacle_pose1 = RigidTransform(RotationMatrix(), [0.01, 0.035, 0.02])  # Adjust position
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("obstacle_link", obstacle1),
    obstacle_pose1)

# obstacle_pose2 = RigidTransform(RotationMatrix(), [-0.01, 0.035, 0.02])  # Adjust position
# plant.WeldFrames(
#     plant.world_frame(),
#     plant.GetFrameByName("obstacle_link", obstacle2),
#     obstacle_pose2)

obstacle_pose3 = RigidTransform(RotationMatrix(), [-0.035, -0.005, 0.02])  # Adjust position
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("obstacle_link", obstacle3),
    obstacle_pose3)

p_GgraspO = [0, 0, .065]
R_GgraspO = RotationMatrix.MakeXRotation(-np.pi / 2)
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("base_wsg", gripper),
    RigidTransform(R_GgraspO, p_GgraspO))

# Fix right finger to left finger
right_finger_joint = plant.GetJointByName("right_finger_sliding_joint", gripper)
left_finger_joint = plant.GetJointByName("left_finger_sliding_joint", gripper)

# Set default joint translation to 0.025
# right_finger_joint.set_default_translation(0.025)
left_finger_joint.set_default_translation(-0.025)

plant.Finalize()

print("Number of positions: ", plant.num_positions())

# Cell 3: Initialize the CIrisPlantVisualizer
q_star = np.zeros(plant.num_positions())

# The object we will use to perform our certification
cspace_free_polytope = CspaceFreePolytope(
    plant, 
    scene_graph,
    SeparatingPlaneOrder.kAffine,
    q_star)

visualizer = CIrisPlantVisualizer(
    plant,
    builder,
    scene_graph,
    cspace_free_polytope,
    viz_role=Role.kIllustration,
    allow_plus_3dof=True
)


visualizer.task_space_diagram.ForcedPublish(visualizer.task_space_diagram_context)

INFO:drake:Meshcat listening for connections at http://localhost:7002


Number of positions:  3


### 1.1. Use Sliders to Visualize the Scene in Meshcat

In [4]:
sliders = []

plant_context = visualizer.plant_context
diagram = visualizer.task_space_diagram
diagram_context = visualizer.task_space_diagram_context

for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]
    q_high = plant.GetPositionUpperLimits()[i]
    step = (q_high - q_low) / 100
    sliders.append(widgets.FloatSlider(
        min=q_low, max=q_high, 
        value=0, step=step, 
        description=f"q{i}"))
    
# q = np.zeros(plant.num_positions())
q = [-3.14, -0.78, -0.03]
def handle_slider_change(change, idx):
    q[idx] = change['new']
    # print(visualizer.check_collision_q_by_ik(q))
    plant.SetPositions(plant_context, q)
    diagram.ForcedPublish(diagram_context)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

FloatSlider(value=0.0, description='q0', max=3.14, min=-3.14, step=0.06280000000000001)

FloatSlider(value=0.0, description='q1', max=1.0, min=-1.0, step=0.02)

FloatSlider(value=0.0, description='q2', max=0.0, min=-0.055, step=0.00055)

## 1.2. Setup Grasping and Placement Space

In [5]:
# Define the 8 corner points of the convex hull
x_bounds = [-3.14, 3.14]
y_bounds = [-1, 1]
z_bounds = [-0.055, -0.024]

lower_joint_limits = np.array([x_bounds[0], y_bounds[0], z_bounds[0]])
upper_joint_limits = np.array([x_bounds[1], y_bounds[1], z_bounds[1]])

z_bounds_grasp = [-0.025, -0.024]

# Generate all corner points
placement_points = np.array([[x, y, z] for x in x_bounds for y in y_bounds for z in z_bounds])
grasp_points = np.array([[x, y, z] for x in x_bounds for y in y_bounds for z in z_bounds_grasp])
###### For 2 dimensional example:
# placement_points = np.array([[x, y] for x in x_bounds for y in z_bounds])
# grasp_points = np.array([[x, y] for x in x_bounds for y in z_bounds_grasp])

# Compute the convex hull
placement_hull = ConvexHull(placement_points)
grasp_hull = ConvexHull(grasp_points)

# Convert ConvexHull to HPolyhedron
def convex_hull_to_hpolyhedron(hull):
    A = hull.equations[:, :-1]
    b = -hull.equations[:, -1]
    return HPolyhedron(A, b)

placement_polytope = convex_hull_to_hpolyhedron(placement_hull)
grasp_polytope = convex_hull_to_hpolyhedron(grasp_hull)

# 2. Manipulation Planner Class

In [6]:
class ManipulationPlanner():

    def __init__(self, 
            visualizer: CIrisPlantVisualizer,
            CP: HPolyhedron,
            CG: HPolyhedron,
            max_grasps: int = 20,
        ):
        self.builder = visualizer.builder
        self.plant = visualizer.plant
        self.plant_context = visualizer.plant_context
        self.scene_graph = visualizer.scene_graph
        self.q_star = visualizer.q_star
        self.rat_fk = visualizer.rat_forward_kin
        self.inspector = visualizer.model_inspector
        self.diagram = visualizer.task_space_diagram
        self.diagram_context = visualizer.task_space_diagram_context
        self.lower_joint_limits = visualizer.q_lower_limits
        self.upper_joint_limits = visualizer.q_upper_limits
        self.visualize_cspace = visualizer.visualize_collision_constraint
        self.cspace_free_polytope = visualizer.cspace_free_polytope
        
        self.CP = CP
        self.CG = CG
        self.max_grasps = max_grasps
        
        self.q_dim = self.plant.num_positions()
        
        self.cs_free = self._generate_cfree()
        
    def set_max_grasps(self, max_grasps: int):
        self.max_grasps = max_grasps
    
    
    def compute_trajectory(self, x_init, x_goal, display=True):
        print("Finding path for minimum grasps")
        path = self._find_minimum_grasp_path(
                x_init = x_init,
                x_goal = x_goal,
                P = self.CP,
                G = self.CG,
                lower_joint_limits = self.lower_joint_limits,
                upper_joint_limits = self.upper_joint_limits,
                c_free_polytopes = self.cs_free
            )
        
        print("Path found. Generating trajectory")
        traj = self._generate_trajectory(path)
        
        print("Trajectory generated. Starting display")
        if display:
            self.display_trajectory(traj)
            
        return path, traj
        
    def display_trajectory(self, traj, meshcat=True, plotly=False):
        if meshcat:
            num_points = int((traj.end_time() - traj.start_time()) * 4000)
            for t in np.linspace(traj.start_time(), traj.end_time(), num_points):
                q = traj.value(t)
                self.plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(self.plant_context, q[2])
                # self.plant.GetJointByName("right_finger_sliding_joint", gripper).set_translation(self.plant_context, q[3])
                self.plant.GetJointByName("base_revolute_joint", gripper).set_angle(self.plant_context, q[1])
                self.plant.GetJointByName("cap_to_base", cap).set_angle(self.plant_context, -q[0])
                self.diagram.ForcedPublish(self.diagram_context)
                time.sleep(0.01)
        path = [traj.value(t) for t in np.linspace(traj.start_time(), traj.end_time(), 100)]
        path_q = [q.ravel() for q in path] # Convert to list of numpy arrays
        if plotly:
            self.visualize_cspace(factor=1, num_points=30, paths=[path_q], filled_polytopes=self.cs_free)
        return path_q
    
    def _generate_trajectory(self, path):
        trajs = []
        print("Generating trajectory from:")
        for q0, q1 in zip(path[:-1], path[1:]):
            print("From\t{}\tto\t{}".format(q0, q1))
            traj, result = self._generate_edge_trajectory(q0, q1)
            if not result.is_success():
                print("failed to generate edge trajectory from {} to {}".format(q0, q1))
                return None
            trajs.append(traj)
        return CompositeTrajectory.AlignAndConcatenate(trajs)
        
        
    def _generate_edge_trajectory(self, x_init, x_goal):
        trajopt = GcsTrajectoryOptimization(self.plant.num_positions())
        gcs_regions = trajopt.AddRegions(self.cs_free, order=1, h_min=0.01)
        source = trajopt.AddRegions([Point(x_init)], order=0)
        target = trajopt.AddRegions([Point(x_goal)], order=0)
        trajopt.AddEdges(source, gcs_regions)
        trajopt.AddEdges(gcs_regions, target)
        trajopt.AddPathLengthCost()
        options = GraphOfConvexSetsOptions()
        [traj, result] = trajopt.SolvePath(source, target, options)
        print(f"result.is_success() = {result.is_success()}")
        return traj, result        
        
    
    def _find_minimum_grasp_path(
            self,
            x_init: np.ndarray,
            x_goal: np.ndarray,
            P: HPolyhedron,
            G: HPolyhedron,
            lower_joint_limits: np.ndarray, 
            upper_joint_limits: np.ndarray,
            c_free_polytopes: list[HPolyhedron]
            ) -> np.ndarray:
            
        for i in range(self.max_grasps):
            if i % 10 == 0 and i > 0:
                print(f"Trying with {i} grasps")
                
            path = self._solve_for_n_grasps_CC(
                i,
                x_init,
                x_goal,
                P,
                G,
                lower_joint_limits,
                upper_joint_limits,
                c_free_polytopes
            )
            
            if path is not None:
                print(f"Found a solution with {i} grasps")
                return path  
        print(f"No solution found for less than {self.max_grasps} grasps")
    
    
    def _solve_for_n_grasps_CC(
            self,
            n_grasps: int, 
            x_init: np.ndarray,
            x_goal: np.ndarray,
            P: HPolyhedron,
            G: HPolyhedron,
            lower_joint_limits: np.ndarray, 
            upper_joint_limits: np.ndarray,
            c_free_polytopes: list[HPolyhedron]
            ) -> np.ndarray:
        # Initialize the program
        prog = MathematicalProgram()
        n_points = 2 * n_grasps + 2
        n_vars = self.q_dim * n_points
        
        # Create decision variables
        x = prog.NewContinuousVariables(n_vars, "x")
        
        # 1. Cost function: Minimize sum of squared distances between consecutive points
        
        c = np.zeros((n_vars - self.q_dim, 1))
        C = np.zeros((n_vars - self.q_dim, n_vars))
        C[:, self.q_dim:] = np.eye(n_vars - self.q_dim)
        C[:, :-self.q_dim] -= np.eye(n_vars - self.q_dim)
        
        Q, b = self._to_quadratic_form(C, c)
        Q += np.eye(Q.shape[1]) * 1e-6  # Add a small value to the diagonal to make Q positive definite
        
        prog.AddQuadraticCost(Q=Q, b=b, vars=x)
        
        # 2. Initial and goal constraints
        prog.AddLinearEqualityConstraint(np.eye(self.q_dim), x_init.flatten(), x[:self.q_dim])
        prog.AddLinearEqualityConstraint(np.eye(self.q_dim), x_goal.flatten(), x[-self.q_dim:])
        
        
        ###### Placement and Grasping Constraints should be adapted task specific #####
        
        # 3. Placement constraints (equality) -> Cap remains constant in transit paths
        for i in range(n_grasps + 1):
            idx1 = 2*self.q_dim*i
            idx2 = 2*self.q_dim*i + self.q_dim
            prog.AddLinearEqualityConstraint(x[idx1] - x[idx2] == 0)
        
        # 4. Grasp constraints (equality between gripper orientations)
        for i in range(n_grasps):
            idx = 2*self.q_dim*(i+1) - 1
            prog.AddLinearEqualityConstraint(x[idx-self.q_dim+1] - x[idx-self.q_dim+2] - x[idx+1] + x[idx+2] == 0)
        
        ###### end of manual constraints
        
        
        # 5. Inequality Constraints (Placement hull)
        for i in range(n_points):
            prog.AddLinearConstraint(
                P.A(),  # Coefficient matrix
                -np.inf * np.ones_like(P.b()),  # Lower bound
                P.b(),  # Upper bound
                x[self.q_dim*i:self.q_dim*(i+1)]
            )
        # 6. Inequality Constraints (Grasp hull) 
        # for i in range(n_points - 2):
        #     prog.AddLinearConstraint(
        #         G.A(),
        #         -np.inf * np.ones_like(G.b()),
        #         G.b(),
        #         x[self.q_dim*(i+1):self.q_dim*(i+2)]
        #     )
        
        # 7. Joint limits (inequality)
        prog.AddBoundingBoxConstraint(
            np.tile(lower_joint_limits, n_points),
            np.tile(upper_joint_limits, n_points),
            x
        )
        
        # 7. Collision-free polytope constraints (MIP)
        # Ensure all points are contained in at least one polytope
        
        M = 1e6  # Big-M constant (adjust based on problem scale)
        for i in range(n_points):
            x_i = x[self.q_dim*i:self.q_dim*(i+1)]
            
            # Creates one bineary 0-1 variable for each polytope
            z_i = prog.NewBinaryVariables(len(c_free_polytopes), name=f"z_{i}")
            
            # Each point x_i must be contained in at least one polytope
            prog.AddLinearConstraint(sum(z_i) >= 1)
            
            for j, poly in enumerate(c_free_polytopes):
                A_j = poly.A()
                b_j = poly.b()
                z_j = z_i[j]
                
                # Add constraints for each row of the polytope
                for k in range(A_j.shape[0]):
                    # Construct coefficient matrix [A_j_row | M]
                    coeffs = np.hstack([A_j[k], M])
                    # Combine x_i and z_j into variable vector
                    vars = np.hstack([x_i, [z_j]])
                    # A_j @ x_i + M * z[j] <= b_j + M
                    prog.AddLinearConstraint(
                        coeffs,
                        -np.inf,
                        b_j[k] + M,
                        vars
                    )
                
        # 8. Connected components constraints
        connected_components = self._compute_connected_components(P, G, c_free_polytopes)
        K = len(connected_components)
        
        # create binary variables for component membership (excluding init ang goal)
        z = {}
        for i in range (1, n_points-1): # exclude init and goal
            z[i] = prog.NewBinaryVariables(K, f"z_{i}")   
            # Each point must belong to exactly one component
            prog.AddLinearConstraint(sum(z[i]) == 1)
        
        # Consecutive points must belong to the same component, ignoring init and goal
        for i in range(1, n_points-2, 2):
            for k in range(K):
                prog.AddLinearConstraint(z[i][k] == z[i+1][k])
        
        # Enforce that each component contains at least one point
        for i in range(1, n_points-1):
            x_i = x[self.q_dim*i:self.q_dim*(i+1)]
            z_i = z[i]
            
            for k, comp in enumerate(connected_components):
                A_k = comp.A()
                b_k = comp.b()
                z_k = z_i[k]
                
                for j in range(A_k.shape[0]):
                    coeffs = np.hstack([A_k[j], M])
                    vars = np.hstack([x_i, [z_k]])
                    prog.AddLinearConstraint(coeffs, -np.inf, b_k[j] + M, vars)
        
        # Solve the problem
        solver = MosekSolver()
        result = solver.Solve(prog)
        
        if not result.is_success():
            return None
        
        return result.GetSolution(x).reshape(-1, self.q_dim)
    
    @staticmethod
    def _to_quadratic_form(C, c):
        return np.dot(C.T, C), -np.dot(C.T, c).flatten()
    
    def _compute_connected_components(
        self,
        placement: HPolyhedron,
        grasp: HPolyhedron,
        c_free: list[HPolyhedron]
    ) -> list[HPolyhedron]:
        """
        Computes convex connected components as:
        {placement ∩ grasp ∩ (union of c_free)}.
        Returns convex hulls of connected regions.
        """
        components = []
        
        # Compute intersections with each c_free polytope
        for poly in c_free:
            intersection = placement.Intersection(grasp).Intersection(poly)
            if not intersection.IsEmpty():
                components.append(intersection)
        
        # Merge overlapping components
        merged = []
        for comp in components:
            add_to_merged = False
            for m in merged:
                if comp.Intersection(m).IsEmpty():
                    continue
                # Merge overlapping components
                vertices_comp = visualizer.get_polytope_vertices(comp)
                vertices_m = visualizer.get_polytope_vertices(m)
                vertices = np.vstack([vertices_comp, vertices_m])
                merged_hull = ConvexHull(vertices)
                merged_poly = convex_hull_to_hpolyhedron(merged_hull)
                merged.remove(m)
                merged.append(merged_poly)
                add_to_merged = True
                break
            if not add_to_merged:
                merged.append(comp)
        
        return merged
    
    
    def _generate_cfree(self):
        generator = RandomGenerator(1234)
        checker = SceneGraphCollisionChecker(
            model=self.diagram,
            robot_model_instances=[self.plant.GetModelInstanceByName("robot")],
            edge_step_size=0.01,
        )
        options = IrisFromCliqueCoverOptions()
        options.num_points_per_visibility_round = 200
        options.coverage_termination_threshold = 0.99
        options.iris_options.configuration_space_margin = 0.00001
        # See https://github.com/RobotLocomotion/drake/issues/21343 -> If getting this issue reduce c-space margin above
        regions = IrisInConfigurationSpaceFromCliqueCover(checker, options, generator, [])
        return regions
    
    

# 3. Example Usage

In [7]:
planner = ManipulationPlanner(visualizer, placement_polytope, grasp_polytope, 50)

INFO:drake:Allocating contexts to support implicit context parallelism 12
INFO:drake:Using default max clique solver MaxCliqueSolverViaGreedy.
INFO:drake:Current Fraction of Domain Covered = 0
INFO:drake:IrisFromCliqueCover Iteration 1/100
INFO:drake:Clique added to queue. There are 81/200 points left to cover.
INFO:drake:Clique added to queue. There are 41/200 points left to cover.
INFO:drake:Clique added to queue. There are 18/200 points left to cover.
INFO:drake:Clique added to queue. There are 3/200 points left to cover.
INFO:drake:Finished adding cliques. Total of 4 clique added. Number of cliques left to process = 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake: Checking robot::collision against obstacle::collision_obs has already required 50 counter-example searches; still searching...
INFO:drake:IrisInConfigurationSpace: Te

In [ ]:
x_init = np.array([-3.14, -0.78, -0.03])
x_goal = np.array([0., -0.78, -0.03])

path, traj = planner.compute_trajectory(x_init, x_goal)

Finding path for minimum grasps
Found a solution with 6 grasps
Path found. Generating trajectory
Generating trajectory from:
From	[-3.14 -0.78 -0.03  0.03]	to	[-3.14        0.38912427 -0.025       0.02499959]


INFO:drake:Solved GCS shortest path using Mosek with convex_relaxation=true and preprocessing=true and rounding.
INFO:drake:Found 5 unique paths, discarded 0 duplicate paths.
INFO:drake:Finished 5 rounding solutions with Mosek.


result.is_success() = True
From	[-3.14        0.38912427 -0.025       0.02499959]	to	[-2.55167775  0.97744649 -0.02497185  0.02494312]


INFO:drake:Solved GCS shortest path using Mosek with convex_relaxation=true and preprocessing=true and rounding.
INFO:drake:Found 5 unique paths, discarded 3 duplicate paths.
INFO:drake:Finished 5 rounding solutions with Mosek.


result.is_success() = True
From	[-2.55167775  0.97744649 -0.02497185  0.02494312]	to	[-2.55167775  0.38922825 -0.02498622  0.02490596]


...

result.is_success() = True
From	[-0.1985746   0.38912609 -0.025       0.02493925]	to	[ 0.          0.58770068 -0.02499961  0.02499961]


INFO:drake:Solved GCS shortest path using Mosek with convex_relaxation=true and preprocessing=true and rounding.
INFO:drake:Found 5 unique paths, discarded 4 duplicate paths.
INFO:drake:Finished 5 rounding solutions with Mosek.


result.is_success() = True
From	[ 0.          0.58770068 -0.02499961  0.02499961]	to	[ 0.   -0.78 -0.03  0.03]


INFO:drake:Solved GCS shortest path using Mosek with convex_relaxation=true and preprocessing=true and rounding.
INFO:drake:Found 5 unique paths, discarded 0 duplicate paths.
INFO:drake:Finished 5 rounding solutions with Mosek.


result.is_success() = True
Trajectory generated. Starting display


In [ ]:
traj_path = planner.display_trajectory(traj, plotly=True)